<a href="https://colab.research.google.com/github/harenlin/PySpark-Learning/blob/main/Read_Write_and_Validate_Data_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reading, Writing and Validating Data in PySpark HW

Welcome to your first coding homework assignment in PySpark! I hope you enjoyed the lecture on Reading, Writing and Validating dataframes. Now it's time to put what you've learned into action! 

I've included several instructions below to help guide you through this homework assignment which I hope will get you feeling even comfortable reading, writing and validating dataframes. If you get stuck at any point, feel free to jump to the next lecture where I will guide you through my solutions to the HW assignment. 

Have fun!

Let's dig right in!


## But first things first.....
We need to always begin every Spark session by creating a Spark instance. Let's go ahead and use the method we learned in the lecture in the cell below. Also see if you can remember how to open the Spark UI (using a link that automatically guides you there). 

In [ ]:
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('RWV').getOrCreate()
cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
print(spark)

     |████████████████████████████████| 212.4MB 67kB/s 
     |████████████████████████████████| 204kB 17.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=c90f9beae07fa733583ca0aaea71516a4c0df3f020bf13bbc71e6ce14e5ab375
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark
You are working with 1 core(s)


In [ ]:
spark

## Next let's start by reading a basic csv dataset

Download the pga_tour_historical dataset that is attached to this lecture and save it whatever folder you want, then read it in. 

**Data Source:** https://www.kaggle.com/bradklassen/pga-tour-20102018-data

Rememer to try letting Spark infer the header and infer the Schema types!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/My Drive/PySpark/Datasets/'
pga = spark.read.csv(path + 'pga_tour_historical.csv', inferSchema=True, header=True)

Mounted at /content/drive


## 1. View first 5 lines of dataframe
First generate a view of the first 5 lines of the dataframe to get an idea of what is inside. We went over two ways of doing this... see if you can remember BOTH ways. 

In [ ]:
pga.show(5)

+---------------+------+----------------+--------------------+-----+
|    Player Name|Season|       Statistic|            Variable|Value|
+---------------+------+----------------+--------------------+-----+
|Robert Garrigus|  2010|Driving Distance|Driving Distance ...|   71|
|   Bubba Watson|  2010|Driving Distance|Driving Distance ...|   77|
| Dustin Johnson|  2010|Driving Distance|Driving Distance ...|   83|
|Brett Wetterich|  2010|Driving Distance|Driving Distance ...|   54|
|    J.B. Holmes|  2010|Driving Distance|Driving Distance ...|  100|
+---------------+------+----------------+--------------------+-----+
only showing top 5 rows



## 2. Print the schema details

Now print the details of the dataframes schema that Spark infered to ensure that it was infered correctly. Sometimes it is not infered correctly, so we need to watch out!

In [ ]:
pga.printSchema()

root
 |-- Player Name: string (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Statistic: string (nullable = true)
 |-- Variable: string (nullable = true)
 |-- Value: string (nullable = true)



In [ ]:
pga.columns

['Player Name', 'Season', 'Statistic', 'Variable', 'Value']

In [ ]:
pga.describe()

DataFrame[summary: string, Player Name: string, Season: string, Statistic: string, Variable: string, Value: string]

## 3. Edit the schema during the read in

We can see from the output above that Spark did not correctly infer that the "value" column was an integer value. Let's try specifying the schema this time to let spark know what the schema should be.

Here is a link to see a list of PySpark data types in case you need it (also attached to the lecture): 
https://spark.apache.org/docs/latest/sql-ref-datatypes.html

In [ ]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType, DateType
# pyspark data types: https://spark.apache.org/docs/latest/sql-ref-datatypes.html
data_schema = [StructField("Player Name", StringType(), True), \
               StructField("Season", IntegerType(), True), \
               StructField("Statistic", StringType(), True), \
               StructField("Variable", StringType(), True), \
               StructField("Value", IntegerType(), True)]
final_struc = StructType(fields = data_schema)

path = '/content/drive/My Drive/PySpark/Datasets/'
pga = spark.read.csv(path + 'pga_tour_historical.csv', schema = final_struc)
print(pga.printSchema())

root
 |-- Player Name: string (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Statistic: string (nullable = true)
 |-- Variable: string (nullable = true)
 |-- Value: integer (nullable = true)

None


## 4. Generate summary statistics for only one variable

See if you can generate summary statistics for only the "Value" column using the .describe function

(count, mean, stddev, min, max) 

In [ ]:
pga.describe(['Value']).show()

+-------+------------------+
|summary|             Value|
+-------+------------------+
|  count|           1657247|
|   mean|12494.388998743096|
| stddev|157274.75673570752|
|    min|              -178|
|    max|           3564954|
+-------+------------------+



In [ ]:
pga.select('Value').summary("count", "mean", "stddev", "min", "max").show()

+-------+------------------+
|summary|             Value|
+-------+------------------+
|  count|           1657247|
|   mean|12494.388998743096|
| stddev|157274.75673570752|
|    min|              -178|
|    max|           3564954|
+-------+------------------+



## 5. Generate summary statistics for TWO variables
Now try to generate ONLY the count min and max for BOTH the "Value" and "Season" variable using the select. You can't use the .describe function for this one but see if you can remember which function you CAN use. 

In [ ]:
pga.select('Value', 'Season').summary("count", "min", "max").show()

+-------+-------+-------+
|summary|  Value| Season|
+-------+-------+-------+
|  count|1657247|2740403|
|    min|   -178|   2010|
|    max|3564954|   2018|
+-------+-------+-------+



## 6. Write a parquet file

Now try writing a parquet file (not partitioned) from the pga dataset. But first create a new dataframe containing ONLY the the "Season" and "Value" fields (using the "select command you used in the question above) and write a parquet file partitioned by "Season". This is a bit of a challenge aimed at getting you ready for material that will be covered later on in the course. Don't feel bad if you can't figure it out.

*Note that if any of your variable names contain spaces, spark will produce an error message with this call. That is why we are selecting ONLY the "Season" and "Value" fields. Ideally we should renamed those columns but we haven't gotten to that yet in this course but we will soon!*

In [ ]:
df = pga.select('Season', 'Value')
df.write.mode('overwrite').parquet(path + 'partition_parquet/')

## 7. Write a partioned parquet file

You will need to use the same limited dataframe that you created in the previous question to accomplish this task as well. 

In [ ]:
df.write.mode('overwrite').partitionBy('Season').parquet(path + 'partitioned_parquet/')
df.show(5)

+------+-----+
|Season|Value|
+------+-----+
|  null| null|
|  2010|   71|
|  2010|   77|
|  2010|   83|
|  2010|   54|
+------+-----+
only showing top 5 rows



## 8. Read in a partitioned parquet file

Now try reading in the partitioned parquet file you just created above. 

In [ ]:
parquet = spark.read.parquet(path + 'partitioned_parquet/')
#Note: if you add a * to the end of the path, the Season var will be automatically dropped
parquet.show()

+-----+------+
|Value|Season|
+-----+------+
|   71|  2010|
|   77|  2010|
|   83|  2010|
|   54|  2010|
|  100|  2010|
|   63|  2010|
|   88|  2010|
|   64|  2010|
|   64|  2010|
|   92|  2010|
|   75|  2010|
|   54|  2010|
|   76|  2010|
|   94|  2010|
|   82|  2010|
|   85|  2010|
|   79|  2010|
|   89|  2010|
|   88|  2010|
|   91|  2010|
+-----+------+
only showing top 20 rows



## 9. Reading in a set of paritioned parquet files

Now try only reading Seasons 2010, 2011 and 2012.

In [ ]:
# Notice that this method only gives you the "Value" column
partitioned = spark.read.parquet(path+'partitioned_parquet/Season=2010/', path+'partitioned_parquet/Season=2011/', path+'partitioned_parquet/Season=2012/')
partitioned.show(5)

+-----+
|Value|
+-----+
|   71|
|   77|
|   83|
|   54|
|  100|
+-----+
only showing top 5 rows



In [ ]:
# need to use this method, spark.read.option("basePath", path+'partitioned_parquet/'), to get the "Season" and "Value" Columns
dataframe = spark.read.option("basePath", path+'partitioned_parquet/').parquet(path+'partitioned_parquet/Season=2010/', 
                                                                               path+'partitioned_parquet/Season=2011/', 
                                                                               path+'partitioned_parquet/Season=2012/')
dataframe.show(5)

+-----+------+
|Value|Season|
+-----+------+
|   71|  2010|
|   77|  2010|
|   83|  2010|
|   54|  2010|
|  100|  2010|
+-----+------+
only showing top 5 rows



## 10. Create your own dataframe

Try creating your own dataframe below using PySparks *.createDataFrame* function. See if you can make one that contains 4 variables and at least 3 rows. 

Let's see how creative you can get on the content of the dataframe :)

In [ ]:
# initalize list of lists (same as in python)
df = [['Haren', 22, 175, 83], ['Jimmy', 30, 176, 72], ['Andy', 30, 175, 75]]
# create the pandas dataframe
df = spark.createDataFrame(df, ['Name', 'Age', 'Weight', 'Height'])
df.show()

+-----+---+------+------+
| Name|Age|Weight|Height|
+-----+---+------+------+
|Haren| 22|   175|    83|
|Jimmy| 30|   176|    72|
| Andy| 30|   175|    75|
+-----+---+------+------+



## We're done! Great job!